In [19]:
from huggingface_hub import notebook_login

In [20]:
# 需要在huggingface官网绑定相关的token进行认证
notebook_login()

In [21]:
from datasets import Dataset, DatasetDict


# 指向包含 Arrow 文件和其他 JSON 元数据的文件夹
path_to_data = "sr_16000.hf"

dataset = Dataset.load_from_disk(f'{path_to_data}')

# # 加载整个数据集
# dataset = load_dataset('json', data_files={'data': f'{path_to_data}/data-00000-of-00001.arrow'})

# 手动分割数据集，假设我们用 80% 作为训练集，20% 作为测试集
train_test_split = dataset.train_test_split(test_size=0.2)

# 获取训练集和测试集
train_set = train_test_split['train']
test_set = train_test_split['test']

# 查看一些样本
print(train_set[0])
print(test_set[0])

{'sentence': '常州人孕妈，喜欢新款，喜欢爆款，喜欢大象灰，需要发票，在意价格。', 'audio_id': '725104', 'audio': {'path': None, 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.05392456,
       -0.04681396,  0.        ]), 'sampling_rate': 16000}, 'duration': 6.2171875}
{'sentence': '常州人喜欢小包，喜欢monogram，喜欢老花，喜欢onthego喜欢托特。', 'audio_id': '726098', 'audio': {'path': None, 'array': array([0.        , 0.        , 0.        , ..., 0.00531006, 0.01705933,
       0.03060913]), 'sampling_rate': 16000}, 'duration': 7.73225}


In [22]:
dataset_dict = DatasetDict({
    'train': train_set,
    'test': test_set
})
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio_id', 'audio', 'duration'],
        num_rows: 72
    })
    test: Dataset({
        features: ['sentence', 'audio_id', 'audio', 'duration'],
        num_rows: 18
    })
})

In [23]:
dataset_dict = dataset_dict.remove_columns(["audio_id", "duration"])
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 72
    })
    test: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 18
    })
})

In [24]:
from transformers import WhisperTokenizer, WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
from transformers import WhisperProcessor

# 模型待选 openai/whisper-large-v3
processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
print(dataset_dict["train"][0])

{'sentence': '常州人孕妈，喜欢新款，喜欢爆款，喜欢大象灰，需要发票，在意价格。', 'audio': {'path': None, 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.05392456,
       -0.04681396,  0.        ]), 'sampling_rate': 16000}}


In [27]:
input_str = dataset_dict["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input: {input_str}")
print(f"Decoded w/ special: {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal: {input_str == decoded_str}")

Input: 常州人孕妈，喜欢新款，喜欢爆款，喜欢大象灰，需要发票，在意价格。
Decoded w/ special: <|startoftranscript|><|transcribe|><|notimestamps|>常州人孕妈，喜欢新款，喜欢爆款，喜欢大象灰，需要发票，在意价格。<|endoftext|>
Decoded w/out special: 常州人孕妈，喜欢新款，喜欢爆款，喜欢大象灰，需要发票，在意价格。
Are equal: True


In [28]:
def prepare_dataset(batch, feature_extractor, tokenizer):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [29]:
dataset_dict = dataset_dict.map(prepare_dataset, remove_columns=dataset_dict.column_names["train"], num_proc=4, fn_kwargs={'feature_extractor': feature_extractor, 'tokenizer': tokenizer})

Map (num_proc=4):   0%|          | 0/72 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/18 [00:00<?, ? examples/s]

In [30]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [31]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [32]:
import evaluate

metric = evaluate.load("wer")

In [33]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [34]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [35]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [37]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-ATOM", 
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",  # 更新后的新版本参数
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()